In [2]:
!pip install openai==0.28
!pip install line-bot-sdk
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.4/758.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.6 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, abort
from pyngrok import ngrok, conf
from flask_ngrok import run_with_ngrok
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'

conf.get_default().auth_token = "2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz"

app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

# 質問回数を追跡するための変数
user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    # 文字数チェック
    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            # 回答の文字数をチェックして制限
            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    public_url = ngrok.connect(5000)
    app.run()


<ipython-input-32-540e4f86fe28>:16: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-32-540e4f86fe28>:17: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://65c8-34-66-199-129.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


<ipython-input-32-540e4f86fe28>:72: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:38:06] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:38:34] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:39:20] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Aug/2024 15:39:36] "POST / HTTP/1.1" 200 -


In [ ]:
!pip install openai==0.28
!pip install line-bot-sdk
!pip install flask-ngrok
!pip install flask
!pip install pyngrok
!pip install flask pyngrok line-bot-sdk openai

  Using cached line_bot_sdk-3.11.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached aenum-3.1.15-py3-none-any.whl.metadata (3.7 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
Using cached line_bot_sdk-3.11.0-py2.py3-none-any.whl (758 kB)
Using cached aenum-3.1.15-py3-none-any.whl (137 kB)
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached pyngrok-7.2.0-py3-none-any.whl.metadata (7.4 kB)
Using cached pyngrok-7.2.0-py3-none-any.whl (22 kB)


In [ ]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'

app = Flask(__name__)

line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

# 質問回数を追跡するための変数
user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    # 文字数チェック
    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            # 回答の文字数をチェックして制限
            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-11-0ad0ed327c51>:11: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-11-0ad0ed327c51>:12: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug

In [ ]:
!git add .
!git commit -m "Fix path issues and update startCommand"
!git push origin main



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git add .
!git commit -m "Update startCommand to use gunicorn"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git add .
!git commit -m "Configure app to use gunicorn and listen on 0.0.0.0:5000"
!git push origin main



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.models import MessageEvent, TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'MessageEvent' from 'linebot.v3.models' (/usr/local/lib/python3.10/dist-packages/linebot/v3/models/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent, TextMessage, TextSendMessage
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'TextMessage' from 'linebot.v3.webhook' (/usr/local/lib/python3.10/dist-packages/linebot/v3/webhook.py)

In [ ]:

import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage, TextSendMessage
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)



ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhooks import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'WebhookHandler' from 'linebot.v3.webhooks' (/usr/local/lib/python3.10/dist-packages/linebot/v3/webhooks/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent  # 修正されたインポートステートメント
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('d8eb77477635fbc8955c02c5163afaf5')
line_bot_api = MessagingApi(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))
handler = WebhookHandler(os.getenv('d8eb77477635fbc8955c02c5163afaf5'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'linebot.v3.messages'

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent  # 修正されたインポートステートメント
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextMessage, TextSendMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'linebot.v3.messages'

In [ ]:
pip install line-bot-sdk


In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage, TextSendMessage
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))


user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.message import TextMessage, TextSendMessage  # 修正されたインポートステートメント
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'linebot.v3.message'

In [ ]:
!pip install --upgrade line-bot-sdk

In [ ]:
!pip install openai==0.28
!pip install flask-ngrok
!pip install flask
!pip install pyngrok

In [3]:
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
import openai

app = Flask(__name__)

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'
line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-3-759db7b9ed9e>:10: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Lx65hwBaeVtUtyz4DLeWXGke+1Np3yUyBQ0yPh3LK9DETzVXHp3LaBUHIOVYR0K5/3+skVNUXR9h9Mp2ouYZGMmhgAJQ/6fvYU3kCUhfnp+FrOt43YDSSXwff9h2m5SMPdPiSddnoLwYIzHxYqWgIQdB04t89/1O/w1cDnyilFU=')
<ipython-input-3-759db7b9ed9e>:11: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('e0069f76b523e833fae449753218f7e1')
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on

In [4]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage, TextSendMessage
from linebot.v3.webhooks import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
import openai

app = Flask(__name__)

openai.api_key = 'sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc'
line_bot_api = MessagingApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)



ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [5]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage  # TextSendMessageを削除
from linebot.v3.webhooks import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.models import TextSendMessage  # ここからTextSendMessageをインポート
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('sk-proj-0rWegtKf1k8b1H5jiy9qT3BlbkFJFH3IhU8ZAQVEftyw71Sc')
line_bot_api = MessagingApi(os.getenv('2jdIrcNwCn5YCQTpbiYqkBLSD7V_3BQF7fQt46HLxM3bbeDJz'))
handler = WebhookHandler(os.getenv('e0069f76b523e833fae449753218f7e1'))


user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ImportError: cannot import name 'WebhookHandler' from 'linebot.v3.webhooks' (/usr/local/lib/python3.10/dist-packages/linebot/v3/webhooks/__init__.py)

In [6]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi, TextMessage  # TextSendMessageを削除
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import TextSendMessage  # ここからTextSendMessageをインポート
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('OPENAI_API_KEY')
line_bot_api = MessagingApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)


ImportError: cannot import name 'TextSendMessage' from 'linebot.v3.messaging' (/usr/local/lib/python3.10/dist-packages/linebot/v3/messaging/__init__.py)

In [ ]:
import os
from flask import Flask, request, abort
from linebot.v3.messaging import MessagingApi
from linebot.v3.webhook import WebhookHandler, MessageEvent
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messages import TextSendMessage, TextMessage
import openai

app = Flask(__name__)

# 環境変数からAPIキーを取得
openai.api_key = os.getenv('OPENAI_API_KEY')
line_bot_api = MessagingApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))

user_question_count = {}

@app.route("/", methods=['POST'])
def home():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id
    user_message = event.message.text

    if len(user_message) > 250:
        reply_message = "ご質問は250文字以内でお願いします！"
    else:
        if user_id not in user_question_count:
            user_question_count[user_id] = 0

        if user_question_count[user_id] < 3:
            system_instruction = "以下の質問に対して、回答を日本語で250文字以内にまとめてください。"

            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_instruction},
                    {"role": "user", "content": user_message}
                ],
            )
            reply_message = response.choices[0].message['content']

            if len(reply_message) > 250:
                reply_message = reply_message[:250] + '...'

            user_question_count[user_id] += 1
        else:
            reply_message = "貴重なお時間をいただき、誠にありがとうございました。回答は３問までです！お会いできる日を心待ちにしております！"

    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=reply_message)
    )

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)
